In [6]:
import numpy as np
import pandas as pd

# Fixer la graine pour la reproductibilité
np.random.seed(42)

# Charger le fichier existant
df = pd.read_csv('Final1.csv')

# Vérifier et convertir la colonne datetime au format datetime
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['datetime'])  # Supprimer les lignes avec des dates invalides si nécessaire

# Filtrer les utilisateurs uniques existants
existing_users = df['user_number'].unique()

# Créer un DataFrame pour les utilisateurs avec anomalies
user_anomalies_df = pd.DataFrame()

# Période de la coupure d'eau (8 heures)
anomaly_duration = pd.Timedelta(hours=8)

# Ajouter des anomalies pour 10 utilisateurs aléatoires
for _ in range(20):
    # Générer un user_number unique aléatoire qui n'existe pas dans les utilisateurs existants
    user_number = np.random.randint(1000, 10000)
    
    # Vérifier si l'user_number existe déjà et en générer un nouveau si nécessaire
    while user_number in existing_users:
        user_number = np.random.randint(1000, 10000)
    
    # Sélectionner les données pour un utilisateur existant aléatoire
    user_data = df[df['user_number'] == np.random.choice(existing_users)].copy()
    
    # Modifier le user_number pour cet utilisateur
    user_data['user_number'] = user_number
    
    # Nombre d'anomalies à ajouter
    num_anomalies = 1  # Exemple : 1 anomalie
    
    # Trouver les indices où la consommation est égale à 50 ou 70
    anomaly_indices = user_data[user_data['Consommation'].isin([50, 70])].index
    num_anomalies = min(num_anomalies, len(anomaly_indices))
    
    if num_anomalies > 0:
        # Sélectionner aléatoirement les indices où l'on va ajouter des anomalies
        selected_anomaly_indices = np.random.choice(anomaly_indices, size=num_anomalies, replace=False)

        # Ajouter les anomalies
        for idx in selected_anomaly_indices:
            anomaly_start = user_data.loc[idx, 'datetime']
            anomaly_end = anomaly_start + anomaly_duration
            
            # Sélectionner les lignes dans cette plage de 8 heures
            anomaly_range = (user_data['datetime'] >= anomaly_start) & (user_data['datetime'] < anomaly_end)
            
            # Appliquer la coupure d'eau : réduire la consommation à 0
            user_data.loc[anomaly_range, 'Consommation'] = 0  # Coupure d'eau (consommation à 0)
            
            # Marquer ces lignes comme anomalies et définir le type d'anomalie
            user_data.loc[anomaly_range, ['anomalie', 'type']] = [1, 'Coupure d\'eau']
    
    # Ajouter les données modifiées pour cet utilisateur au DataFrame principal
    user_anomalies_df = pd.concat([user_anomalies_df, user_data], ignore_index=True)

# Ajouter les utilisateurs avec anomalies dans la base initiale
final_df = pd.concat([df, user_anomalies_df], ignore_index=True)

# Exporter le DataFrame final mis à jour vers un fichier CSV
final_df.to_csv('Final2.csv', index=False)

# Confirmation de l'exportation
print("Les anomalies de coupure d'eau ont été ajoutées et exportées vers 'Final2.csv'.")


C:\Users\MSI\AppData\Local\Temp\ipykernel_8056\2764576871.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Final1.csv')


Les anomalies de coupure d'eau ont été ajoutées et exportées vers 'Final2.csv'.
